<a href="https://colab.research.google.com/github/JayPBhatia/MastersProject/blob/main/ForReport_SmartGrid_ADSC_16850_DataSet_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [29]:
#connect to G Drive 
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
%cd drive/MyDrive/ForReport/

[Errno 2] No such file or directory: 'drive/MyDrive/ForReport/'
/content/drive/MyDrive/ForReport


In [31]:
import numpy as np
from numpy import array
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [33]:
from matplotlib.backends.backend_pdf import PdfPages

def _draw_as_table(df, pagesize):
    alternating_colors = [['white'] * len(df.columns), ['lightgray'] * len(df.columns)] * len(df)
    alternating_colors = alternating_colors[:len(df)]
    fig, ax = plt.subplots(figsize=pagesize)
    ax.axis('tight')
    ax.axis('off')
    the_table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=df.columns,
                        rowColours=['lightblue']*len(df),
                        colColours=['lightblue']*len(df.columns),
                        cellColours=alternating_colors,
                        loc='center')
    return fig
  

def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                fig.text(0, 0,
                         "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                         ha='center', fontsize=12)
            fig.suptitle(filename+"_describe")
            pdf.savefig(fig, bbox_inches='tight')
            plt.show()
            plt.close()

In [34]:
def adfuller_test(series, signif=0.05):
  x = adfuller(series, autolag='AIC')
  output = {' P-value': x[1], 
              f'isStationary (P Value <  {signif})': x[1] < signif }
            
  return pd.Series(output)


In [35]:
extention = ".png"

def plt_save(plt, printName, plt_name):
    title = printName + "_" + plt_name
    print(title)
    suptitle = plt.suptitle(title, y=1.02)
    plt.tight_layout()
    plt.savefig( title + extention, bbox_extra_artists=(suptitle,), bbox_inches='tight')
    plt.show()

def processDataFile(base_url, folder_name, file_name, start_col, end_col):
    file_path = base_url+folder_name + "/csv/" +file_name + ".csv"
    printName = folder_name.replace("/", "_") + "_" + file_name
    print(f"processing {printName}")
    df = pd.read_csv(file_path)
    df = df.iloc[:, start_col:end_col]

    print(df.info())
    for col in df.columns:
      print(f"{printName} {col} unique values {df[col].unique().size}" )
    print(df.describe())
    dataframe_to_pdf(df.describe(),printName + '_describe.pdf')
  
    df.plot(subplots=True,figsize=(20,20))
    plt_save(plt, printName, 'plot' )

    axes = df.hist(layout=(1,len(df.columns)), figsize=(35,5))
    for ax, col in zip(axes[0], df.columns):  # For each column...
        ax.axvline(df[col].mean(), c='k')  # Plot mean
        ax.axvline(df[col].median(), c='r')  # Plot median 
    plt_save(plt, printName, 'hist' )
    
    sns.pairplot(df, diag_kind='kde')
    plt_save(plt, printName, 'pairplot' )

    plt.figure(figsize=(20,20))
    cor = df.corr()
    sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
    plt_save(plt, printName, 'corr' )

    
    print("aduffler")
    data = df.apply(lambda x: adfuller_test(x), axis=0)
    fig, ax = plt.subplots()
    fig.patch.set_visible(False)
    table = ax.table(cellText=data.values, colLabels=data.columns, loc='center', cellLoc='center')
    ax.set_title(file_name + " aduffler test isStationary (P Value < 0.05)'")
    ax.axis("off") 
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.auto_set_column_width(col=list(range(len(df.columns)))) 
    plt.savefig( printName + '_aduffler.png')
    plt.show()

    
    fig, axes = plt.subplots(nrows=1, ncols=len(df.columns),figsize=(40,5))
    for col, axe in zip(df.columns,axes):
        plot_acf(df[col], title=col, ax=axe);
    plt_save(plt, printName, 'acf' )

    
    fig, axes = plt.subplots(nrows=1, ncols=len(df.columns),figsize=(40,5))
    for col, axe in zip(df.columns,axes):
        plot_pacf(df[col], title=col, ax=axe);
    plt_save(plt, printName, 'pacf' )

## EDA

In [36]:
bied_file = "https://raw.githubusercontent.com/smartgridadsc/IEC61850SecurityDataset/master/Disturbance/Breaker%20Failure/csv/BIED100.csv"
bied_df = pd.read_csv(bied_file)

tied_file = "https://raw.githubusercontent.com/smartgridadsc/IEC61850SecurityDataset/master/Disturbance/Breaker%20Failure/csv/TIED23.csv"
tied_df = pd.read_csv(tied_file)

lied_file = "https://raw.githubusercontent.com/smartgridadsc/IEC61850SecurityDataset/master/Disturbance/Breaker%20Failure/csv/LIED10.csv"
lied_df = pd.read_csv(lied_file)



In [37]:
bied_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 16 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Circuit breaker Open/Close Status                  600 non-null    int64  
 1    Disconnector Open/Close Status                    600 non-null    int64  
 2    Earth switch Open/Close Status                    600 non-null    int64  
 3    Protection system healthy (Trip circuit healthy)  600 non-null    int64  
 4    Control level - Local or remote                   600 non-null    bool   
 5    Protection tripped                                600 non-null    bool   
 6    Circuit breaker mechanical failure                600 non-null    int64  
 7    Auxiliary power failure                           600 non-null    bool   
 8    Current Line 'L1'                                 600 non-null    int64  
 9    Current L

In [38]:
for col in bied_df.columns:
  print(f"{col} unique values {bied_df[col].unique().size}" )

Circuit breaker Open/Close Status unique values 1
 Disconnector Open/Close Status unique values 1
 Earth switch Open/Close Status unique values 1
 Protection system healthy (Trip circuit healthy) unique values 1
 Control level - Local or remote unique values 1
 Protection tripped unique values 2
 Circuit breaker mechanical failure unique values 1
 Auxiliary power failure unique values 1
 Current Line 'L1' unique values 11
 Current Line 'L2' unique values 11
 Current Line 'L3' unique values 11
 Voltage Phase 'L1-N' unique values 31
 Voltage Phase 'L2-N' unique values 31
 Voltage Phase 'L3-N' unique values 31
 Frequency unique values 11
 Power Factor unique values 5


In [39]:
bied_df2 = bied_df.iloc[:, 8:16]
bied_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0    Current Line 'L1'     600 non-null    int64  
 1    Current Line 'L2'     600 non-null    int64  
 2    Current Line 'L3'     600 non-null    int64  
 3    Voltage Phase 'L1-N'  600 non-null    int64  
 4    Voltage Phase 'L2-N'  600 non-null    int64  
 5    Voltage Phase 'L3-N'  600 non-null    int64  
 6    Frequency             600 non-null    float64
 7    Power Factor          600 non-null    float64
dtypes: float64(2), int64(6)
memory usage: 37.6 KB


In [49]:
bied_df.tail()

,Circuit breaker Open/Close Status,Disconnector Open/Close Status,Earth switch Open/Close Status,Protection system healthy (Trip circuit healthy),Control level - Local or remote,Protection tripped,Circuit breaker mechanical failure,Auxiliary power failure,Current Line 'L1',Current Line 'L2',Current Line 'L3',Voltage Phase 'L1-N',Voltage Phase 'L2-N',Voltage Phase 'L3-N',Frequency,Power Factor
595,0,1,0,1,False,True,1,False,5,4,-2,38100,38098,38095,49.95,0.84
596,0,1,0,1,False,True,1,False,1,4,-3,38120,38090,38118,50.02,0.86
597,0,1,0,1,False,True,1,False,-1,-4,3,38092,38104,38100,49.97,0.86
598,0,1,0,1,False,True,1,False,2,-4,-5,38104,38091,38110,50.02,0.86
599,0,1,0,1,False,True,1,False,1,4,-5,38098,38111,38108,50.05,0.87


In [40]:
lied_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Circuit breaker Open/Close Status                  600 non-null    int64  
 1    Disconnector Open/Close Status                    600 non-null    int64  
 2    Earth switch Open/Close Status                    600 non-null    int64  
 3    Protection system healthy (Trip circuit healthy)  600 non-null    int64  
 4    Control level - Local or remote                   600 non-null    bool   
 5    Protection tripped                                600 non-null    bool   
 6    Circuit breaker mechanical failure                600 non-null    int64  
 7    Auxiliary power failure                           600 non-null    bool   
 8    Intertrip command send                            600 non-null    bool   
 9    Intertrip

In [41]:
for col in lied_df.columns:
  print(f"{col} unique values {lied_df[col].unique().size}" )

Circuit breaker Open/Close Status unique values 2
 Disconnector Open/Close Status unique values 1
 Earth switch Open/Close Status unique values 1
 Protection system healthy (Trip circuit healthy) unique values 1
 Control level - Local or remote unique values 1
 Protection tripped unique values 2
 Circuit breaker mechanical failure unique values 1
 Auxiliary power failure unique values 1
 Intertrip command send unique values 2
 Intertrip command receive unique values 1
 Current Line 'L1' unique values 8
 Current Line 'L2' unique values 10
 Current Line 'L3' unique values 11
 Voltage Phase 'L1-N' unique values 10
 Voltage Phase 'L2-N' unique values 10
 Voltage Phase 'L3-N' unique values 11
 Active Power unique values 12
 Reactive power unique values 12
 Frequency unique values 9
 Power Factor unique values 5


In [47]:
lied_df.tail()

,Circuit breaker Open/Close Status,Disconnector Open/Close Status,Earth switch Open/Close Status,Protection system healthy (Trip circuit healthy),Control level - Local or remote,Protection tripped,Circuit breaker mechanical failure,Auxiliary power failure,Intertrip command send,Intertrip command receive,Current Line 'L1',Current Line 'L2',Current Line 'L3',Voltage Phase 'L1-N',Voltage Phase 'L2-N',Voltage Phase 'L3-N',Active Power,Reactive power,Frequency,Power Factor
595,0,1,0,1,False,True,1,False,True,False,0,0,0,0,0,0,0,0,0.0,0.0
596,0,1,0,1,False,True,1,False,True,False,0,0,0,0,0,0,0,0,0.0,0.0
597,0,1,0,1,False,True,1,False,True,False,0,0,0,0,0,0,0,0,0.0,0.0
598,0,1,0,1,False,True,1,False,True,False,0,0,0,0,0,0,0,0,0.0,0.0
599,0,1,0,1,False,True,1,False,True,False,0,0,0,0,0,0,0,0,0.0,0.0


In [42]:
lied_df2 = lied_df.iloc[:, 10:20]
lied_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0    Current Line 'L1'     600 non-null    int64  
 1    Current Line 'L2'     600 non-null    int64  
 2    Current Line 'L3'     600 non-null    int64  
 3    Voltage Phase 'L1-N'  600 non-null    int64  
 4    Voltage Phase 'L2-N'  600 non-null    int64  
 5    Voltage Phase 'L3-N'  600 non-null    int64  
 6    Active Power          600 non-null    int64  
 7    Reactive power        600 non-null    int64  
 8    Frequency             600 non-null    float64
 9    Power Factor          600 non-null    float64
dtypes: float64(2), int64(8)
memory usage: 47.0 KB


In [43]:
tied_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 21 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Circuit breaker Open/Close Status                  600 non-null    int64  
 1    Disconnector Open/Close Status                    600 non-null    int64  
 2    Earth switch Open/Close Status                    600 non-null    int64  
 3    Protection system healthy (Trip circuit healthy)  600 non-null    int64  
 4    Control level - Local or remote                   600 non-null    bool   
 5    Protection tripped                                600 non-null    bool   
 6    Circuit breaker mechanical failure                600 non-null    int64  
 7    transformer mechanical failure (over temp)        600 non-null    bool   
 8    Transformer mechanical failure (over pressure)    600 non-null    bool   
 9    Tap chang

In [44]:
for col in tied_df.columns:
  print(f"{col} unique values {tied_df[col].unique().size}" )

Circuit breaker Open/Close Status unique values 1
 Disconnector Open/Close Status unique values 1
 Earth switch Open/Close Status unique values 1
 Protection system healthy (Trip circuit healthy) unique values 1
 Control level - Local or remote unique values 1
 Protection tripped unique values 1
 Circuit breaker mechanical failure unique values 1
 transformer mechanical failure (over temp) unique values 1
 Transformer mechanical failure (over pressure) unique values 1
 Tap changer mechanical failure unique values 1
 Auxiliary power failure unique values 1
 Current Line 'L1' unique values 23
 Current Line 'L2' unique values 23
 Current Line 'L3' unique values 24
 Voltage Phase 'L1-N' unique values 31
 Voltage Phase 'L2-N' unique values 31
 Voltage Phase 'L3-N' unique values 31
 Active Power unique values 530
 Reactive power unique values 518
 Frequency unique values 11
 Power Factor unique values 5


In [50]:
tied_df.tail()

,Circuit breaker Open/Close Status,Disconnector Open/Close Status,Earth switch Open/Close Status,Protection system healthy (Trip circuit healthy),Control level - Local or remote,Protection tripped,Circuit breaker mechanical failure,transformer mechanical failure (over temp),Transformer mechanical failure (over pressure),Tap changer mechanical failure,...,Current Line 'L1',Current Line 'L2',Current Line 'L3',Voltage Phase 'L1-N',Voltage Phase 'L2-N',Voltage Phase 'L3-N',Active Power,Reactive power,Frequency,Power Factor
595,1,1,0,1,False,False,1,False,False,False,...,315,313,322,38097,38098,38104,30000586,18501148,49.97,0.84
596,1,1,0,1,False,False,1,False,False,False,...,314,321,310,38096,38090,38099,30000362,18501888,50.00,0.86
597,1,1,0,1,False,False,1,False,False,False,...,323,317,314,38115,38100,38109,30000016,18501401,49.99,0.86
598,1,1,0,1,False,False,1,False,False,False,...,321,315,322,38093,38117,38090,30000348,18501952,50.00,0.87
599,1,1,0,1,False,False,1,False,False,False,...,313,317,310,38116,38102,38112,30001947,18501512,49.95,0.83


In [45]:
tied_df2 = tied_df.iloc[:, 11:21]
tied_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0    Current Line 'L1'     600 non-null    int64  
 1    Current Line 'L2'     600 non-null    int64  
 2    Current Line 'L3'     600 non-null    int64  
 3    Voltage Phase 'L1-N'  600 non-null    int64  
 4    Voltage Phase 'L2-N'  600 non-null    int64  
 5    Voltage Phase 'L3-N'  600 non-null    int64  
 6    Active Power          600 non-null    int64  
 7    Reactive power        600 non-null    int64  
 8    Frequency             600 non-null    float64
 9    Power Factor          600 non-null    float64
dtypes: float64(2), int64(8)
memory usage: 47.0 KB


In [46]:
%%script false

#'Normal/Variable_Loading' , 'Normal/No_Variable_Loading', 
list_of_folders = ['Disturbance/Breaker%20Failure', 'Disturbance/Busbar%20Protection', 'Disturbance/Under%20frequency' ]
list_of_tied_files = ['TIED13', 'TIED23']
list_of_bied_files = ['BIED100']
list_of_lied_files = ['LIED10','LIED11', 'LIED12', 'LIED20' , 'LIED21', 'LIED22', 'LIED30', 'LIED31', 'LIED32', 'LIED33', 'LIED40', 'LIED41', 'LIED42', 'LIED43']
base_url = "https://raw.githubusercontent.com/smartgridadsc/IEC61850SecurityDataset/master/"


for folder_name in list_of_folders:
  for file_name in list_of_tied_files:
    processDataFile(base_url, folder_name, file_name, 11, 21)

  for file_name in list_of_bied_files:
    processDataFile(base_url, folder_name, file_name, 8, None)

  for file_name in list_of_lied_files:
    processDataFile(base_url, folder_name, file_name, 10, None)
